In [1]:
!git clone https://github.com/facebookresearch/ParlAI.git

Cloning into 'ParlAI'...
remote: Enumerating objects: 39168, done.
remote: Counting objects: 100% (402/402), done.
remote: Compressing objects: 100% (298/298), done.
remote: Total 39168 (delta 154), reused 271 (delta 98), pack-reused 38766
Receiving objects: 100% (39168/39168), 64.97 MiB | 30.10 MiB/s, done.
Resolving deltas: 100% (27897/27897), done.


In [2]:
!pip install flask_ngrok
!pip install PyYAML==5.1

     |████████████████████████████████| 276kB 7.7MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44074 sha256=982fab6f14afd5073a51dec53068ec629b413753796257dceaeeb8188de53441
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!cd ParlAI; python setup.py develop

running develop
running egg_info
creating parlai.egg-info
writing parlai.egg-info/PKG-INFO
writing dependency_links to parlai.egg-info/dependency_links.txt
writing entry points to parlai.egg-info/entry_points.txt
writing requirements to parlai.egg-info/requires.txt
writing top-level names to parlai.egg-info/top_level.txt
writing manifest file 'parlai.egg-info/SOURCES.txt'
adding license file 'LICENSE' (matched pattern 'LICEN[CS]E*')
writing manifest file 'parlai.egg-info/SOURCES.txt'
running build_ext
Creating /usr/local/lib/python3.7/dist-packages/parlai.egg-link (link to .)
Adding parlai 1.2.0 to easy-install.pth file
Installing parlai script to /usr/local/bin

Installed /content/ParlAI
Processing dependencies for parlai==1.2.0
Searching for jsonlines
Reading https://pypi.org/simple/jsonlines/
Best match: jsonlines 2.0.0
Processing jsonlines-2.0.0-py3-none-any.whl
Installing jsonlines-2.0.0-py3-none-any.whl to /usr/local/lib/python3.7/dist-packages
Adding jsonlines 2.0.0 to easy-inst

In [4]:
import pexpect
class ChatBot:
    def __init__(self):
        self.child = pexpect.spawn('python ParlAI/parlai/scripts/interactive.py -t blended_skill_talk -mf zoo:blender/blender_90M/model', timeout=None)
        self.child.expect('Enter Your Message:')
        self.personality = self.child.before.decode('utf-8', 'ignore').split('[context]')[1]
    def get_response(self): 
        response = (self.child.before.split(b'1m'))[1].split(b'\x1b')
        return response[0].decode('utf-8')
    def send_request(self, req):
        self.child.sendline(req)
        self.child.expect('Enter Your Message:')

In [5]:
subreddits = ["Depression", "CPTSD", "Anxiety", "BorderlinePDisorder",
              "OCD", "Agoraphobia", "schizophrenia", "domesticviolence"]

In [13]:
from flask import *
import requests
import pickle
from flask_ngrok import run_with_ngrok
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



keyid = 0
maxlen = 1000
alpha = 0.125
num_words = 100000
oov_token = '<UNK>'
pad_type = 'post'
trunc_type = 'post'


tokenizer = None
with open('/content/drive/MyDrive/Colab Notebooks/tokenizerM.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/SDetector')

In [14]:
def predict(msg):
    test_sequences = tokenizer.texts_to_sequences([msg])
    test_padded = pad_sequences(test_sequences, padding=pad_type, truncating=trunc_type, maxlen=maxlen)
    ans = model.predict(test_padded)
    return ans[0]

In [ ]:
from flask import *
import requests
import pickle
from flask_ngrok import run_with_ngrok
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

tokenizer = None
with open('/content/drive/MyDrive/Colab Notebooks/tokenizerM.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/SDetector')

app = Flask(__name__)
run_with_ngrok(app)
global keyid

app.route('/', methods=['POST','GET'])
def home():
    incoming_msg = request.values.get('Body', '').lower()
    session[keyid] = {'msg_history':"",'id':keyid, 'question':0, 'bot': ChatBot()}
    retval = keyid
    keyid+=1
    return str(keyid)

app.route('/bot',method=['POST','GET'])
def bot():
    incoming_msg = request.values.get('Body', '').lower()
    sess_id,text = incoming_msg.split('###')
    sess_id = int(sess_id)
    msg_history = session[sess_id]['msg_history']

    if incoming_msg == 'clear':
        msg_history = ''
        incoming_msg = 'hi'
    if(len(msg_history) + len(incoming_msg)) <= maxlen:
        msg_history+= '. ' + incoming_msg
    else:
        msg_history = incoming_msg      
    session[sess_id]['msg_history'] = msg_history
    if incoming_msg == 'done':
        session.pop(session[sess_id][id])
        return "Thank you for using the app!"

    scores = predict(msg_history)
    most_pred = subreddits[np.argmax(scores)]
    return_msg = ""

    if session[sess_id][question]!=0 and session[sess_id][question]%5 == 0:
        return_msg = f"You are showing symptoms of {most_pred} according to our analysis. If you are satisfied with it you can write 'done'. Else write 'okay' to continue."
        return return_msg

    else:
        session[sess_id]['bot'].send_request(incoming_msg)
        return_msg = session[sess_id]['bot'].get_response()
        return return_msg




    